# Download data and get list of addresses to geocode

We'll use the [Richmond Citywide Traffic Crashes](https://www.transparentrichmond.org/Police-Department/Richmond-Police-Department-Citywide-Traffic-Crashe/af5y-3epb) dataset.

In [ ]:
import pandas as pd
import requests

In [ ]:
# Create a new folder called 'inputs' if it doesn't already exist.
# Using an exclamation point emulates a Terminal command.
!mkdir -p inputs

url = 'https://www.transparentrichmond.org/api/views/af5y-3epb/rows.csv?fourfour=af5y-3epb&accessType=DOWNLOAD'
r = requests.get(url, allow_redirects=False)

# Save file to inputs folder
open('inputs/richmond_crashes.csv', 'wb').write(r.content)

In [ ]:
richmond_crashes = pd.read_csv('inputs/richmond_crashes.csv', parse_dates=['collision_date'])
richmond_crashes.tail()

## Clean dupes?

In [ ]:
dupes = richmond_crashes[
    richmond_crashes['report_number'].duplicated()
].reset_index(drop=True)
dupes_reports = dupes['report_number'].to_list()
dupes_reports

In [ ]:
richmond_crashes[
    richmond_crashes['report_number'].isin(dupes_reports)
]

I don't really feel like cleaning this right now, so for the purpose of this lecture (geocoding!), we're going to skip doing that.

## Create timestamp column

In [ ]:
# Convert `collision_date` to string (to concatenate with collision_time)
richmond_crashes['collision_date'] = richmond_crashes['collision_date'].astype(str)

In [ ]:
richmond_crashes

In [ ]:
# check for NaN in collision_date or collision_time
richmond_crashes[
    richmond_crashes['collision_date'].isna() |
    richmond_crashes['collision_time'].isna()
]

In [ ]:
# fill those NaNs with midnight
richmond_crashes['collision_time'] = richmond_crashes['collision_time'].fillna('00:00:00')

In [ ]:
# create timestamp column (this will be a string/object data type)
richmond_crashes['timestamp'] = richmond_crashes['collision_date'] +  ' ' + richmond_crashes['collision_time']

# convert timestamp column to datetime data type
richmond_crashes['timestamp'] = pd.to_datetime(richmond_crashes['timestamp'])

In [ ]:
richmond_crashes.tail()

In [ ]:
# check one of the NaN rows
# the index below may change depending on the data
richmond_crashes.iloc[674]

In [ ]:
# filter out records that don't have addresses
geocodable = richmond_crashes[
    richmond_crashes['primary_road'].notnull() |
    richmond_crashes['secondary_road'].notnull()
].reset_index(drop=True)

In [ ]:
# create intersection column
geocodable['intersection'] = geocodable['primary_road'] + ' AND ' + geocodable['secondary_road'] + ', RICHMOND, CA'

In [ ]:
geocodable

## Export geocodable crashes dataset

In [ ]:
# Create an outputs folder
!mkdir -p outputs

In [ ]:
geocodable.to_csv('outputs/geocodable.csv', index=False)

## Dedupe intersections

Why? We don't want to geocode the same intersection more than once!

In [ ]:
intersections = geocodable[['intersection']].copy()
intersections

In [ ]:
intersections = intersections.drop_duplicates().reset_index(drop=True)
intersections

We only have 833 intersections to geocode! Awesome.

## Export intersections to geocode

In [ ]:
intersections.to_csv('outputs/intersections.csv', index=False)